# Run the redun workflow

In [ ]:
import lamindb as ln
import lamindb.schema as lns

ln.nb.header()

Before running the workflow on the input files, let us check that we can query for them.

In [ ]:
ln.select(lns.DObject).join(lns.Run).join(lns.Jupynb, id="0ymQDuqM5Lwq").df()

We add the following code block at the end of the workflow:
```
import lndb_setup
import lamindb as ln
import lamindb.schema as lns
from lamin_logger import logger
from pathlib import Path
lndb_setup.load("redun-lamin-fasta")  # load the LaminDB instance we'd like to use
pipeline = ln.select(lns.Pipeline, name="lamin-redun-fasta").one()    
run = lns.Run(
    name="Test run", pipeline_id=pipeline.id, pipeline_v=pipeline.v
)
logger.info(f"Loaded pipeline {pipeline}")
logger.info(f"Created run {run}")
# Ingest pipeline outputs
ingest = ln.Ingest(run)
for filepath in Path("./data/").glob("results.tgz"):  # User needs to decide what to track, here just results.tgz  # noqa
    ingest.add(filepath)
ingest.commit()
# Link pipeline inputs
inputs = ln.select(lns.DObject).join(lns.Run).join(lns.Jupynb, id="0ymQDuqM5Lwq").all()
links = [lns.RunIn(dobject_id=dobject.id, run_id=run.id) for dobject in inputs]
ln.add(links)
```

Let us now call the workflow:

In [ ]:
!redun run workflow.py main --input-dir fasta/

## View the LaminDB content

In [ ]:
ln.view()